# k-Nearest Neighbors

The model for kNN is the entire training dataset. When a prediction is required for a unseen data instance, the kNN algorithm will search through the training dataset for the k-most similar instances. The prediction attribute of the most similar instances is summarized and returned as the prediction for the unseen instance.

The similarity measure is dependent on the type of data. For real-valued data, the Euclidean distance can be used. Other other types of data such as categorical or binary data, Hamming distance can be used.

In the case of regression problems, the average of the predicted attribute may be returned. In the case of classification, the most prevalent class may be returned.

## Exercise 1 - Explore the Data

The test problem we will be using is iris classification. The problem is comprised of 150 observations of iris flowers from three different species. There are 4 measurements of given flowers: sepal length, sepal width, petal length and petal width, all in the same unit of centimeters. The predicted attribute is the species, which is one of setosa, versicolor or virginica.

It is a standard dataset where the species is known for all instances. As such we can split the data into training and test datasets and use the results to evaluate our algorithm implementation. Good classification accuracy on this problem is above 90% correct, typically 96% or better.

You can download the dataset for free from [iris.data](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data), see the resources section for further details.

In [17]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import itertools

from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
% matplotlib inline
iris= pd.read_csv('iris.csv')

In [4]:
print(iris.head())
iris.columns
iris.dropna(how= "all", inplace= True)

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


In [29]:
#X= iris.ix[:, :4].values
#Y= iris.ix[:, 4].values
train, test = train_test_split(iris.values,  test_size=.4)

## Exercise 2 - Build a k-Nearest Neighbors Class

The derivation can be [found here on Wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm). Here is a useful link that explains how to do a [weighted KNN](http://www.csee.umbc.edu/~tinoosh/cmpe650/slides/K_Nearest_Neighbor_Algorithm.pdf).

The general steps are:
- Locate the k most similar data instances from a test instance
    - This is done by calculating the distance from each instance in the data set to the test instance
- Return the predicted class label 
- Return an accuracy measurement

In [82]:
import math
from operator import itemgetter

class KNNClassifier():
    def __init__(self):
        pass

    def euclideanDistance(self, instance1, instance2, length):
        distance= 0
        for x in range(length):            
            distance += math.pow((instance1[x]- instance2[x]), 2)
            
        return math.sqrt(distance)                                   
    
    def getNeighbors(self, trainingSet, testInstance, k):
        distances= []
        length = len(testInstance)-1
        
        for x in range(len(trainingSet)):
            dist = self.euclideanDistance(testInstance, trainingSet[x],length)
            distances.append((trainingSet[x],dist))
        distances.sort(key=itemgetter(1))
                       
        neighbors = []
        for x in range(k):
            neighbors.append(distances[x][0])                            
                       
        return neighbors                       
        
    def getResponse(self, neighbors):
        classVotes ={}                       

        for x in range(len(neighbors)):                           
            response = neighbors[x][-1]
                           
            if response in classVotes:
                classVotes[response] +=1
            else:
                classVotes[response] =1                       
                       
        sortedVotes = sorted(classVotes.items(), key=itemgetter(1), reverse = True)                       
        return sortedVotes[0][0]

    def get_Accuracy (self, testSet, predictions):
        correct=0
    
        for x in range(len(testSet)):    
            if testSet[x][-1] is predictions[x]:
                correct +=1
         
        return correct / float(len(testSet)) *100                    
                       
    def predict(self, trainingSet, testSet, k=3):
        predictions =[]
        
        for x in range(len(testSet)):
            neighbors= self.getNeighbors(trainingSet, testSet[x],k)
            result = self.getResponse(neighbors)
            predictions.append(result)                                         
            print("> Predicted="+repr(result)+  ", actual="+repr(testSet[x][-1]))                                         
        accuracy= self.get_Accuracy(testSet, predictions)                
        print("Accuracy " + repr(accuracy) + "%")

## Exercise 3 - Try it out on the Iris Data Set. 

In [83]:
knn= KNNClassifier()

In [84]:
knn.predict(train, test, 3)

> Predicted='virginica', actual='virginica'
> Predicted='virginica', actual='virginica'
> Predicted='versicolor', actual='versicolor'
> Predicted='setosa', actual='setosa'
> Predicted='setosa', actual='setosa'
> Predicted='setosa', actual='setosa'
> Predicted='versicolor', actual='versicolor'
> Predicted='virginica', actual='virginica'
> Predicted='virginica', actual='virginica'
> Predicted='versicolor', actual='versicolor'
> Predicted='versicolor', actual='versicolor'
> Predicted='versicolor', actual='versicolor'
> Predicted='virginica', actual='virginica'
> Predicted='versicolor', actual='versicolor'
> Predicted='virginica', actual='virginica'
> Predicted='virginica', actual='virginica'
> Predicted='virginica', actual='virginica'
> Predicted='versicolor', actual='versicolor'
> Predicted='versicolor', actual='versicolor'
> Predicted='virginica', actual='virginica'
> Predicted='versicolor', actual='versicolor'
> Predicted='virginica', actual='virginica'
> Predicted='setosa', actual='se

## Exercise 4 - Check via scikit-learn. Plot the decision regions.

In [76]:
from sklearn import neighbors

X_train= train[:,:4]
Y_train= train[:,4]
X_test = test[:,:4]

knn = neighbors.KNeighborsClassifier(n_neighbors=1)
pred = knn.fit(X_train, Y_train).predict(X_test)

In [77]:
# Wow, this works pretty well :)
Y_test= test[:,4]
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(Y_test, pred).T)
print(classification_report(Y_test, pred, digits=3))

[[16  0  0]
 [ 0 19  2]
 [ 0  0 23]]
             precision    recall  f1-score   support

     setosa      1.000     1.000     1.000        16
 versicolor      0.905     1.000     0.950        19
  virginica      1.000     0.920     0.958        25

avg / total      0.970     0.967     0.967        60

